Ищем коинтегрированные пары

In [1]:
import pandas as pd
from itertools import combinations
import numpy as np
import statsmodels.api as sm

from utils import do_adf_regression, do_aeg_regression, calculate_sharpe_ratio

# Константы
correlation_threshold = 0.95
probability_threshold = 95

open_threshold = 0.75
close_threshold = 0.25

In [2]:
tickers = pd.read_parquet('../data/sp500_stocks.parquet')
tickers.set_index('ticker', inplace=True)
tickers

history = pd.read_parquet('../data/sp500_2023-01-01_2025-01-01.parquet')['Adj Close']
combs = pd.DataFrame(combinations(history.columns, 2), columns=['ticker1', 'ticker2'])
print(len(combs))

126253


In [3]:
combs['correlation'] = combs.apply(lambda x: np.corrcoef(history[x['ticker1']], history[x['ticker2']])[0, 1], axis=1)

combs = combs[combs['correlation'] > correlation_threshold]
combs[['ticker1_company', 'ticker1_sector']] = combs['ticker1'].apply(lambda x: pd.Series(tickers[tickers.index == x][['company', 'sector']].values[0]))
combs[['ticker2_company', 'ticker2_sector']] = combs['ticker2'].apply(lambda x: pd.Series(tickers[tickers.index == x][['company', 'sector']].values[0]))
combs.sort_values(by='correlation', ascending=False, inplace=True)
combs = combs[['ticker1', 'ticker1_company', 'ticker1_sector', 'ticker2', 'ticker2_company', 'ticker2_sector', 'correlation']]
combs

,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation
83182,GOOG,Alphabet Inc. (Class C),Communication Services,GOOGL,Alphabet Inc. (Class A),Communication Services,0.999940
77737,FOX,Fox Corporation (Class B),Communication Services,FOXA,Fox Corporation (Class A),Communication Services,0.998343
114007,NWS,News Corp (Class B),Communication Services,NWSA,News Corp (Class A),Communication Services,0.995214
124585,TRGP,Targa Resources,Energy,WMB,Williams Companies,Energy,0.989555
106066,MCO,Moody's Corporation,Financials,SPGI,S&P Global,Financials,0.989390
...,...,...,...,...,...,...,...
29196,BKNG,Booking Holdings,Consumer Discretionary,V,Visa Inc.,Financials,0.950058
32988,BX,Blackstone Inc.,Financials,NFLX,Netflix,Communication Services,0.950049
87091,HIG,Hartford (The),Financials,RSG,Republic Services,Industrials,0.950035
94790,IT,Gartner,Information Technology,SPG,Simon Property Group,Real Estate,0.950035


Определяем коинтеграция по Augmented Dickey-Fuller

In [4]:
combs['adf_probability'] = combs.apply(lambda x: do_adf_regression(history, x['ticker1'], x['ticker2']), axis=1)
combs

,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation,adf_probability
83182,GOOG,Alphabet Inc. (Class C),Communication Services,GOOGL,Alphabet Inc. (Class A),Communication Services,0.999940,99.0
77737,FOX,Fox Corporation (Class B),Communication Services,FOXA,Fox Corporation (Class A),Communication Services,0.998343,99.0
114007,NWS,News Corp (Class B),Communication Services,NWSA,News Corp (Class A),Communication Services,0.995214,NaN
124585,TRGP,Targa Resources,Energy,WMB,Williams Companies,Energy,0.989555,90.0
106066,MCO,Moody's Corporation,Financials,SPGI,S&P Global,Financials,0.989390,99.0
...,...,...,...,...,...,...,...,...
29196,BKNG,Booking Holdings,Consumer Discretionary,V,Visa Inc.,Financials,0.950058,99.0
32988,BX,Blackstone Inc.,Financials,NFLX,Netflix,Communication Services,0.950049,95.0
87091,HIG,Hartford (The),Financials,RSG,Republic Services,Industrials,0.950035,95.0
94790,IT,Gartner,Information Technology,SPG,Simon Property Group,Real Estate,0.950035,95.0


Определяем коинтеграция по Augmented Engle-Granger

In [5]:
combs['aeg_probability'] = combs.apply(lambda x: do_aeg_regression(history, x['ticker1'], x['ticker2']), axis=1)
combs

,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation,adf_probability,aeg_probability
83182,GOOG,Alphabet Inc. (Class C),Communication Services,GOOGL,Alphabet Inc. (Class A),Communication Services,0.999940,99.0,95.0
77737,FOX,Fox Corporation (Class B),Communication Services,FOXA,Fox Corporation (Class A),Communication Services,0.998343,99.0,99.0
114007,NWS,News Corp (Class B),Communication Services,NWSA,News Corp (Class A),Communication Services,0.995214,NaN,NaN
124585,TRGP,Targa Resources,Energy,WMB,Williams Companies,Energy,0.989555,90.0,NaN
106066,MCO,Moody's Corporation,Financials,SPGI,S&P Global,Financials,0.989390,99.0,99.0
...,...,...,...,...,...,...,...,...,...
29196,BKNG,Booking Holdings,Consumer Discretionary,V,Visa Inc.,Financials,0.950058,99.0,95.0
32988,BX,Blackstone Inc.,Financials,NFLX,Netflix,Communication Services,0.950049,95.0,90.0
87091,HIG,Hartford (The),Financials,RSG,Republic Services,Industrials,0.950035,95.0,90.0
94790,IT,Gartner,Information Technology,SPG,Simon Property Group,Real Estate,0.950035,95.0,90.0


In [6]:
print('All combinations:', len(combs))
combs = combs[(combs['adf_probability'].notna()) | (combs['aeg_probability'].notna())]
combs = combs[(combs['adf_probability'] >= probability_threshold) & (combs['aeg_probability'] >= probability_threshold)]
combs.sort_values(by=['aeg_probability', 'adf_probability', 'correlation'], ascending=False, inplace=True)
print('All combinations after filtering:', len(combs))
combs

All combinations: 1517
All combinations after filtering: 433


,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation,adf_probability,aeg_probability
77737,FOX,Fox Corporation (Class B),Communication Services,FOXA,Fox Corporation (Class A),Communication Services,0.998343,99.0,99.0
106066,MCO,Moody's Corporation,Financials,SPGI,S&P Global,Financials,0.989390,99.0,99.0
112998,NRG,NRG Energy,Utilities,TT,Trane Technologies,Industrials,0.988238,99.0,99.0
96856,JPM,JPMorgan Chase,Financials,KKR,KKR & Co.,Financials,0.987268,99.0,99.0
87514,HLT,Hilton Worldwide,Consumer Discretionary,KKR,KKR & Co.,Financials,0.985304,99.0,99.0
...,...,...,...,...,...,...,...,...,...
32890,BX,Blackstone Inc.,Financials,IBM,IBM,Information Technology,0.951070,95.0,95.0
63393,DVA,DaVita,Health Care,WAB,Wabtec,Industrials,0.951051,95.0,95.0
63254,DVA,DaVita,Health Care,NRG,NRG Energy,Utilities,0.950741,95.0,95.0
87034,HIG,Hartford (The),Financials,NVDA,Nvidia,Information Technology,0.950712,95.0,95.0


In [7]:
live = pd.read_parquet('../data/sp500_2025-01-01_2025-07-01.parquet')['Adj Close']

# calculate_sharpe_ratio(history, "FOX", "FOXA", open_threshold, close_threshold)

combined_index = history.index.union(live.index)
positions = pd.DataFrame(index=combined_index)

if not isinstance(positions.columns, pd.MultiIndex):
    positions.columns = pd.MultiIndex.from_product([[], positions.columns])

def calculate_sharpe_ratios(row, history, live):
    global positions

    ticker1 = row['ticker1']
    ticker2 = row['ticker2']

    training_set = history[[ticker1, ticker2]].copy()
    live_set = live[[ticker1, ticker2]].copy()

    hedge_ration = sm.OLS(training_set[ticker1], training_set[ticker2]).fit().params

    (training_set_sharpe, _, _, training_positions) = calculate_sharpe_ratio(training_set, ticker1, ticker2, hedge_ration.iloc[0], open_threshold, close_threshold)
    (live_set_sharpe, spread_mean, spread_std, live_positions) = calculate_sharpe_ratio(live_set, ticker1, ticker2, hedge_ration.iloc[0], open_threshold, close_threshold)

    combined = pd.concat([training_positions, live_positions])
    positions[training_positions.columns] = combined
    positions = positions.copy()

    return pd.Series((hedge_ration.iloc[0], training_set_sharpe, live_set_sharpe, spread_mean, spread_std))

def to_positions_column(row, names):
    ticker1 = row['ticker1']
    ticker2 = row['ticker2']

    names.append(f"{ticker1}_{ticker2}")

# calculate_sharpe_ratios(combs.iloc[0], history, live)
combs[['hedge_ratio', 'training_set_sharpe', 'live_set_sharpe', 'spread_mean', 'spread_std']] = combs.apply(lambda x: calculate_sharpe_ratios(x, history, live), axis=1)


In [8]:
filtered = combs[(combs['training_set_sharpe'] > 2) & (combs['live_set_sharpe'] > 3.5)]
filtered.sort_values(by=['live_set_sharpe', 'training_set_sharpe'], ascending=False, inplace=True)
print('All combinations after calculating sharpe ratios:', len(filtered))
filtered

All combinations after calculating sharpe ratios: 7


/var/folders/y6/2_hgjshx1x90x58mj93ymhm40000gn/T/ipykernel_80134/1718245924.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.sort_values(by=['live_set_sharpe', 'training_set_sharpe'], ascending=False, inplace=True)


,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation,adf_probability,aeg_probability,hedge_ratio,training_set_sharpe,live_set_sharpe,spread_mean,spread_std
77112,FIS,Fidelity National Information Services,Financials,FITB,Fifth Third Bancorp,Financials,0.959536,95.0,95.0,2.016683,2.104056,4.458191,-3.151111,6.959765
113911,NVR,"NVR, Inc.",Consumer Discretionary,SHW,Sherwin-Williams,Materials,0.970597,99.0,95.0,24.272809,2.001565,4.351045,-1060.447355,411.071683
15802,ANET,Arista Networks,Information Technology,KKR,KKR & Co.,Financials,0.977130,99.0,99.0,0.725637,2.515257,4.071896,1.709350,5.850847
22988,AXP,American Express,Financials,BSX,Boston Scientific,Health Care,0.973786,99.0,95.0,3.143274,2.136791,3.845734,-30.124182,18.848071
60061,DLR,Digital Realty,Real Estate,PNR,Pentair,Industrials,0.952991,99.0,95.0,1.759455,3.051837,3.830391,-4.070781,5.885738
117636,PHM,PulteGroup,Consumer Discretionary,RSG,Republic Services,Industrials,0.965800,95.0,95.0,0.580929,2.116179,3.667558,-32.522091,13.128317
110482,MSI,Motorola Solutions,Information Technology,WELL,Welltower,Real Estate,0.982721,99.0,99.0,3.645609,2.601464,3.638210,-99.401665,47.540381


In [9]:
pairs = np.array(filtered[['ticker1', 'ticker2', 'hedge_ratio', 'spread_mean', 'spread_std']])
pairs

array([['FIS', 'FITB', 2.0166827274862578, -3.1511109715915553,
        6.9597648631874955],
       ['NVR', 'SHW', 24.272809334325157, -1060.4473553353364,
        411.0716827760592],
       ['ANET', 'KKR', 0.7256367325304199, 1.709350418853501,
        5.850847056252798],
       ['AXP', 'BSX', 3.1432737857287134, -30.12418201008268,
        18.848070994096062],
       ['DLR', 'PNR', 1.7594553702250173, -4.070780723144448,
        5.885738451992167],
       ['PHM', 'RSG', 0.5809289971147701, -32.522091052644015,
        13.12831723615651],
       ['MSI', 'WELL', 3.6456087790516287, -99.40166493362965,
        47.540380960504955]], dtype=object)

In [10]:
names = []
filtered.apply(lambda x: to_positions_column(x, names), axis=1)

positions = positions[names]
positions

FIS_FITB      NVR_SHW     ANET_KKR     AXP_BSX     DLR_PNR      \
                FIS FITB     NVR SHW     ANET KKR     AXP BSX     DLR PNR   
Date                                                                        
2023-01-03       -1    1       1  -1        0   0       0   0      -1   1   
2023-01-04       -1    1       1  -1        1  -1       0   0      -1   1   
2023-01-05        0    0       1  -1        1  -1       0   0      -1   1   
2023-01-06        0    0       1  -1        1  -1       0   0      -1   1   
2023-01-09        0    0       1  -1        1  -1       0   0      -1   1   
...             ...  ...     ...  ..      ...  ..     ...  ..     ...  ..   
2025-06-24        0    0       0   0        0   0       0   0       0   0   
2025-06-25        0    0       0   0        0   0       0   0       0   0   
2025-06-26        0    0       0   0        0   0       0   0       1  -1   
2025-06-27        0    0       0   0        0   0       0   0       1  -1   
2025-06-30        0    0       0   0        0   0       0   0       0   0   

           PHM_RSG     MSI_WELL       
               PHM RSG      MSI WELL  
Date                                  
2023-01-03       1  -1       -1    1  
2023-01-04       1  -1       -1    1  
2023-01-05       1  -1       -1    1  
2023-01-06       1  -1       -1    1  
2023-01-09       1  -1       -1    1  
...            ...  ..      ...  ...  
2025-06-24       0   0        1   -1  
2025-06-25       0   0        1   -1  
2025-06-26       0   0        1   -1  
2025-06-27       0   0        1   -1  
2025-06-30       0   0        1   -1  

[624 rows x 14 columns]

In [11]:
count_nonzero = (positions != 0).sum(axis=1) / 2
print(f"Mean amount of open positions {count_nonzero.mean()} with std: {count_nonzero.std()}")


Mean amount of open positions 3.233974358974359 with std: 1.5075198044530653


Что нужно сделать дальше:
 - понять как расчитывается PnL чтобы добавить туда транзакционные расходы
 - посчитать все Sharpe ratio с учетом транзакционных расходов
 - отфильтровать какой-то набор пар с которыми буду работать
 - на основе массива positions подготовить сигналы